In [1]:
from policy import NEATProperty, PropertyArray, properties_to_json
from cib import CIB
from pib import PIB, NEATPolicy

# Application Request

We consider an application that would like to open a new TCP connection using NEAT to a destination host `d1` with the IP `10.1.23.45`. Further, if possible, the MTU of this connection should be greater than 1500 bytes. Finally, the application specifies a `low_latency` profile.

We define three properties to represent these application requirements and combine them into a NEATRequest: 

In [2]:
property1 = NEATProperty(('low_latency', True), precedence=NEATProperty.IMMUTABLE)
property2 = NEATProperty(('remote_ip', '10.1.23.45'), precedence=NEATProperty.IMMUTABLE)
property3 = NEATProperty(('MTU', {"start":1500, "end":9000}), precedence=NEATProperty.OPTIONAL) 
property4 = NEATProperty(('TCP', True))  # OPTIONAL is the default property precedence

request = PropertyArray(property1,property2,property3,property4)

print(request)

├─[low_latency|True]──(MTU|1500.0-9000.0)──[remote_ip|10.1.23.45]──(TCP|True)─┤


# Policy Manager/NEAT logic API

Application requirements from the NEAT logic are passed to the Policy Manager using JSON. For the example above the JSON string could be:

In [3]:
properties_to_json(request)

'{"MTU": {"value": {"end": 9000.0, "start": 1500.0}}, "TCP": {"value": true}, "low_latency": {"precedence": 2, "value": true}, "remote_ip": {"precedence": 2, "value": "10.1.23.45"}}'

# Exemplary Setup

Consider a host with three local interfaces `en0`, `en1`, `ra0`. Two of the interfaces, `en0`, `en1`, are wired while `ra0` is a 3G interface. We populate an instance of the Characteristic Information Base (CIB) with some information about the host interfaces and the network.

In [4]:
cib = CIB('cib/example/')

[INFO]: checking for CIB updates...
[INFO]: new CIB node cib/example/16d095c33b1b43f5b2f26f5aab8fb638.cib. loading...
[ERROR]: Unable to load CIB node cib/example/16d095c33b1b43f5b2f26f5aab8fb638.cib: CIB node is expired
[INFO]: new CIB node cib/example/606b9ee77840011a1d0e409e37e2f51f.cib. loading...
[ERROR]: Unable to load CIB node cib/example/606b9ee77840011a1d0e409e37e2f51f.cib: CIB node is expired
[INFO]: new CIB node cib/example/A.local. loading...
[INFO]: new CIB node cib/example/B.connection. loading...
[INFO]: new CIB node cib/example/be075f299bdcedc73e8de8642433e7a5.cib. loading...
[ERROR]: Unable to load CIB node cib/example/be075f299bdcedc73e8de8642433e7a5.cib: CIB node is expired
[INFO]: new CIB node cib/example/C.connection. loading...
[INFO]: new CIB node cib/example/ee3d9b332849672849ca060af78e6426.cib. loading...
[ERROR]: Unable to load CIB node cib/example/ee3d9b332849672849ca060af78e6426.cib: CIB node is expired
[INFO]: new CIB node cib/example/eec643c70542146eebaf92

The currently known network chracteristics are stored as entries in the CIB, where each entry contains a set of properties associated with some interface:

In [5]:
cib.dump()

═════════════════════════════════════════ CIB START ═════════════════════════════════════════
  0. ├─[capacity|1000]──[interface|en1]──[local_ip|10.3.1.1]──(MTU|1500)─┤
  1. ├─[capacity|10000]──[interface|en0]──[local_ip|10.2.2.2]──(MTU|50.0-9000.0)──[remote_ip|10:54:1.23]──[remote_port|80]──[transport|TCP]₊₁.₀──(utilization|0.63)─┤
  2. ├─[capacity|10000]──[interface|en0]──[local_ip|10.2.1.1]──(MTU|50.0-9000.0)──[remote_ip|10:54:1.23]──[remote_port|80]──[transport|TCP]₊₁.₀──(utilization|0.63)─┤
  3. ├─[capacity|10000]──[interface|en0]──[local_ip|10.2.2.2]──(MTU|50.0-9000.0)──[remote_ip|10:54:1.23]──[remote_port|80]──[transport|TCP]₊₁.₀──(utilization|0.63)─┤
  4. ├─[capacity|10000]──[interface|en0]──[local_ip|10.2.1.1]──(MTU|50.0-9000.0)──[remote_ip|10:54:1.23]──[remote_port|80]──[transport|TCP]₊₁.₀──(utilization|0.63)─┤
  5. ├─[capacity|100]──[interface|en0]──(MTU|1500)─┤
  6. ├─(capacity|1000)──[remote_ip|10:54:1.23]──[remote_port|80]──[transport|TCP]₊₁.₀──(utilization|0.63)─┤
  7. ├

# PIB 

We create one repository for system profiles, and one for policies:

In [6]:
profiles = PIB('pib/example/')
pib = PIB('pib/example/')

[INFO]: Loading policy pib/example/23423.policy...
[INFO]: Loading policy pib/example/b.policy...
[INFO]: Loading policy pib/example/bulk.policy...
[INFO]: Loading policy pib/example/default.profile...
[INFO]: Loading policy pib/example/google.policy...
[INFO]: Loading policy pib/example/low_latency.profile...
[INFO]: Loading policy pib/example/mptcp_so.policy...
[INFO]: Loading policy pib/example/sctp.policy...
[INFO]: Loading policy pib/example/secure.profile...
[INFO]: Loading policy pib/example/tcp_opt.policy...
[INFO]: Loading policy pib/example/tcp_so.policy...
[INFO]: Loading policy pib/example/test.policy.policy...
[INFO]: Loading policy pib/example/test2.pol.policy...
[INFO]: Loading policy pib/example/test333.policy...
[INFO]: Loading policy pib/example/transport_reliable.profile...
[INFO]: Loading policy pib/example/transport_unknown.profile...
[INFO]: Loading policy pib/example/23423.policy...
[INFO]: Loading policy pib/example/b.policy...
[INFO]: Loading policy pib/example

For the current scenario, the low latency profile is defined as follows:

In [14]:
profile1 = NEATPolicy()

profile1.match.add(NEATProperty(('low_latency', True)))
profile1.properties.add(NEATProperty(('iw_wired', True)), 
                        NEATProperty(('interface_latency', (0,40)), precedence=NEATProperty.IMMUTABLE))

profiles.register(profile1)

Next, we define two sample policies and add them to the Policy Information Base (PIB).

A "bulk transfer" policy is configured which is triggered by a specific destination IP, which is known to be the address of backup NFS share:

In [15]:
policy1 = NEATPolicy()

policy1.match.add(NEATProperty(('remote_ip', '10.1.23.45')))
policy1.properties.add(NEATProperty(('capacity', (10000, 100000)), precedence=NEATProperty.IMMUTABLE), 
                       NEATProperty(('MTU', 9600)))

print(policy1)

  0. f2b13a9ce5967411c6dd23ff9e684d6e ├─(remote_ip|10.1.23.45)─┤  ⟶  ╠═[[capacity|10000.0-100000.0]]══[(MTU|9600)]═╣


Another policy is in place to enable TCP window scaling on 10G links (if possible):

In [16]:
policy2 = NEATPolicy(name='TCP options')
policy2.match.insert(NEATProperty(('MTU', 9600)), NEATProperty(('is_wired', True)))
policy2.properties.insert(NEATProperty(('TCP_window_scale', True)))

TypeError: __init__() got an unexpected keyword argument 'name'

In [ ]:
pib.register(policy1)
pib.register(policy2)
pib.dump()

# Lookup Result


## Profile Lookup

First, we apply the `low_latency` profile to the request properties. The `low_latency` property in the request is replaced by the corresponding profile properties:

In [ ]:
print(request.properties)
profiles._lookup(request.properties, remove_matched=True, apply=True)
print(request.properties)

## CIB Lookup

Next a lookup in the CIB is performed. Our NEAT request yields three candidates:

In [ ]:
cib.lookup(request)
request.dump()

Each candidate is comprised of the union of the properties of a single CIB entry and the application request. Whenever  the two sets intersect, the values of the corresponding properties are compared. If two properties match, the associated candidate property score is increased (e.g., `[MTU|1500]+1.0` indicates a new score of 1.0). The score is decreased if there is a mismatch in the property values.

## PIB Lookup
In the next step the policies are applied. The "Bulk transfer" policy is applied first as it posesses the *smallest* number of match entries.

In [ ]:
pib.lookup_all(request.candidates)
request.dump()

Candidate 1 becomes:        

In [ ]:
request.candidates[0].dump()

Next we examine Candidate 2:

In [ ]:
request.candidates[1].dump()

Note that the score of the MTU property was reduced, as it did not match the requested property of the "Bulk transfer" policy.

The "TCP options" policy is not applied as the candidate does not match the policy's MTU property.

---

The third candidate was invalidated because the "Bulk transfer" policy contains an immutable property requiring a capacity of 10G, which candidate 3 cannot fulfil.

---

Finally, we can obtain the total score of the properties associated with each candidate:

In [ ]:
print(request.candidates[0].score)

In [ ]:
print(request.candidates[1].score)

The score indicates that candidate one (interface `en0`) is most suitable for the given application request.

# NEAT Logic
The two candidates can now be passed on to the NEAT logic as JSON strings:

In [ ]:
request.candidates[0].properties.json()

In [ ]:
request.candidates[1].properties.json()

Note that properties which were not matched/updated during the lookup contain `NaN` as a score. This means that the PM did not have enough information to rank theses properties. The NEAT logic must decide how to deal with these unprocessed properties.